In [1]:
!pip install tigramite

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.6/309.6 kB 5.5 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
from tigramite import plotting as tp
import matplotlib.pyplot as plt
from pathlib import Path
from scipy import stats

In [13]:

# Define Nonlinear Equations (Ensuring U has only 2 edges)

def get_nonlinear_equations(n_vars, max_lag):
    """
    Return a list of equation strings describing the structural equations
    for each X_i[t], plus the confounder U[t]. We ensure that U has only 2 edges.
    """
    if n_vars == 4:
        if max_lag == 2:
            return [
                "X4[t] = 0.25 * cos(X1[t-2] * pi/2) + 0.3 * U[t] + trend4[t] + season4[t] + e4",
                "X3[t] = 0.35 * (X4[t])^2 + trend3[t] + season3[t] + e3",
                "X2[t] = 0.3 * sin(X3[t-1] * pi/2) + trend2[t] + season2[t] + e2",
                "X1[t] = 0.4 * (X2[t])^3 + 0.5 * U[t] + trend1[t] + season1[t] + e1",
                "U[t] = eU"
            ]
        elif max_lag == 3:
            return [
                "X4[t] = 0.25 * cos(X1[t-2] * pi/2) + 0.3 * U[t] + trend4[t] + season4[t] + e4",
                "X3[t] = 0.35 * (X4[t])^2 + 0.2 * cos(X2[t-3] * pi/2) + trend3[t] + season3[t] + e3",
                "X2[t] = 0.3 * sin(X3[t-1] * pi/2) + trend2[t] + season2[t] + e2",
                "X1[t] = 0.4 * (X2[t])^3 + 0.5 * U[t] + trend1[t] + season1[t] + e1",
                "U[t] = eU"
            ]
        elif max_lag == 4:
            return [
                "X4[t] = 0.25 * cos(X1[t-4] * pi/2) + 0.3 * U[t] + trend4[t] + season4[t] + e4",
                "X3[t] = 0.35 * (X4[t])^2 + 0.2 * cos(X2[t-3] * pi/2) + trend3[t] + season3[t] + e3",
                "X2[t] = 0.3 * sin(X3[t-1] * pi/2) + trend2[t] + season2[t] + e2",
                "X1[t] = 0.4 * (X2[t])^3 + 0.5 * U[t] + trend1[t] + season1[t] + e1",
                "U[t] = eU"
            ]

    elif n_vars == 6:
        # Keep U->X6, U->X1
        if max_lag == 2:
            return [
                "X6[t] = 0.85 * sin(X5[t] * pi/2) + 0.4 * U[t] + trend6[t] + season6[t] + e6",
                "X5[t] = 0.4 * cos(X4[t-1] * pi/2) + trend5[t] + season5[t] + e5",
                "X4[t] = 0.25 * cos(X1[t-2] * pi/2) + 0.3 * sin(X5[t-1] * pi/2) + trend4[t] + season4[t] + e4",
                "X3[t] = 0.35 * (X4[t])^2 + trend3[t] + season3[t] + e3",
                "X2[t] = 0.3 * sin(X3[t-1] * pi/2) + trend2[t] + season2[t] + e2",
                "X1[t] = 0.4 * (X2[t])^3 + 0.5 * U[t] + trend1[t] + season1[t] + e1",
                "U[t] = eU"
            ]
        elif max_lag == 3:
            return [
                "X6[t] = 0.85 * sin(X5[t] * pi/2) + 0.4 * U[t] + trend6[t] + season6[t] + e6",
                "X5[t] = 0.4 * cos(X4[t-1] * pi/2) + trend5[t] + season5[t] + e5",
                "X4[t] = 0.25 * cos(X1[t-2] * pi/2) + trend4[t] + season4[t] + e4",
                "X3[t] = 0.35 * (X4[t])^2 + 0.2 * cos(X2[t-3] * pi/2) + trend3[t] + season3[t] + e3",
                "X2[t] = 0.3 * sin(X3[t-1] * pi/2) + trend2[t] + season2[t] + e2",
                "X1[t] = 0.4 * (X2[t])^3 + 0.5 * U[t] + trend1[t] + season1[t] + e1",
                "U[t] = eU"
            ]
        elif max_lag == 4:
            return [
                "X6[t] = 0.85 * sin(X5[t] * pi/2) + 0.4 * U[t] + trend6[t] + season6[t] + e6",
                "X5[t] = 0.4 * cos(X4[t-1] * pi/2) + trend5[t] + season5[t] + e5",
                "X4[t] = 0.25 * cos(X1[t-4] * pi/2) + trend4[t] + season4[t] + e4",
                "X3[t] = 0.35 * (X4[t])^2 + 0.2 * cos(X2[t-3] * pi/2) + trend3[t] + season3[t] + e3",
                "X2[t] = 0.3 * sin(X3[t-1] * pi/2) + trend2[t] + season2[t] + e2",
                "X1[t] = 0.4 * (X2[t])^3 + 0.5 * U[t] + trend1[t] + season1[t] + e1",
                "U[t] = eU"
            ]

    elif n_vars == 8:
        # Keep U->X8, U->X1
        if max_lag == 2:
            return [
                "X8[t] = 0.4 * sin(X7[t] * pi/2) + 0.35 * U[t] + trend8[t] + season8[t] + e8",
                "X7[t] = 0.35 * cos(X6[t-1] * pi/2) + trend7[t] + season7[t] + e7",
                "X6[t] = 0.45 * sin(X5[t] * pi/2) + trend6[t] + season6[t] + e6",
                "X5[t] = 0.4 * cos(X4[t-1] * pi/2) + trend5[t] + season5[t] + e5",
                "X4[t] = 0.25 * cos(X1[t-2] * pi/2) + 0.3 * sin(X5[t-1] * pi/2) + trend4[t] + season4[t] + e4",
                "X3[t] = 0.35 * (X4[t])^2 + trend3[t] + season3[t] + e3",
                "X2[t] = 0.3 * sin(X3[t-1] * pi/2) + trend2[t] + season2[t] + e2",
                "X1[t] = 0.4 * (X2[t])^3 + 0.5 * U[t] + trend1[t] + season1[t] + e1",
                "U[t] = eU"
            ]
        elif max_lag == 3:
            return [
                "X8[t] = 0.4 * sin(X7[t] * pi/2) + 0.35 * U[t] + trend8[t] + season8[t] + e8",
                "X7[t] = 0.35 * cos(X6[t-1] * pi/2) + trend7[t] + season7[t] + e7",
                "X6[t] = 0.45 * sin(X5[t] * pi/2) + trend6[t] + season6[t] + e6",
                "X5[t] = 0.4 * cos(X4[t-1] * pi/2) + trend5[t] + season5[t] + e5",
                "X4[t] = 0.25 * cos(X1[t-2] * pi/2) + trend4[t] + season4[t] + e4",
                "X3[t] = 0.35 * (X4[t])^2 + 0.2 * cos(X2[t-3] * pi/2) + trend3[t] + season3[t] + e3",
                "X2[t] = 0.3 * sin(X3[t-1] * pi/2) + trend2[t] + season2[t] + e2",
                "X1[t] = 0.4 * (X2[t])^3 + 0.5 * U[t] + trend1[t] + season1[t] + e1",
                "U[t] = eU"
            ]
        elif max_lag == 4:
            return [
                "X8[t] = 0.4 * sin(X7[t] * pi/2) + 0.35 * U[t] + trend8[t] + season8[t] + e8",
                "X7[t] = 0.35 * cos(X6[t-1] * pi/2) + trend7[t] + season7[t] + e7",
                "X6[t] = 0.45 * sin(X5[t] * pi/2) + trend6[t] + season6[t] + e6",
                "X5[t] = 0.4 * cos(X4[t-1] * pi/2) + trend5[t] + season5[t] + e5",
                "X4[t] = 0.25 * cos(X1[t-4] * pi/2) + trend4[t] + season4[t] + e4",
                "X3[t] = 0.35 * (X4[t])^2 + 0.2 * cos(X2[t-3] * pi/2) + trend3[t] + season3[t] + e3",
                "X2[t] = 0.3 * sin(X3[t-1] * pi/2) + trend2[t] + season2[t] + e2",
                "X1[t] = 0.4 * (X2[t])^3 + 0.5 * U[t] + trend1[t] + season1[t] + e1",
                "U[t] = eU"
            ]

    # If no match, return empty
    return []


# Nonlinear Time Series Generator Class

class NonlinearTimeSeriesGenerator:
    def __init__(
        self,
        noise_scale=0.1,
        trend_strength=0.01,
        seasonal_strength=0.5,
        seasonal_period=12,
        random_state=None
    ):
        self.noise_scale = noise_scale
        self.trend_strength = trend_strength
        self.seasonal_strength = seasonal_strength
        self.seasonal_period = seasonal_period
        self.random_state = random_state
        if random_state is not None:
            np.random.seed(random_state)

    def generate_noise(self, size):
        return np.random.normal(0, self.noise_scale, size=size)

    def generate_trend(self, n_points, var_idx):
        trend_modifier = (var_idx + 1) * 0.5
        t = np.arange(n_points)
        return self.trend_strength * trend_modifier * t

    def generate_seasonality(self, n_points, var_idx):
        phase_shift = 2 * np.pi * var_idx / 8
        t = np.arange(n_points)
        season1 = np.sin(2 * np.pi * t / self.seasonal_period + phase_shift)
        season2 = 0.5 * np.cos(4 * np.pi * t / self.seasonal_period + phase_shift)
        return self.seasonal_strength * (season1 + season2)

    def evaluate_term(self, term, var_values, X, U, t):
        """Safely parse and evaluate one term, e.g. '0.3 * sin(X2[t-1] * pi/2)'."""
        parts = term.split('*')
        coef = float(parts[0].strip())
        expr = parts[1].strip()

        # Helper to parse lag safely
        def parse_lag(lag_str):
            if 't-' in lag_str:
                # e.g., "t-2"
                return int(lag_str.split('-')[1])
            # if there's no '-', default to 0
            return 0

        # 1) Check for confounder U
        if expr.startswith('U['):
            # e.g. "U[t]" or "U[t-1]"
            if expr == 'U[t]':
                return coef * U[t]
            else:
                # e.g. "U[t-2]"
                lagval = parse_lag(expr.split('[')[1].split(']')[0])
                if t - lagval >= 0:
                    return coef * U[t - lagval]
                else:
                    return 0.0

        # 2) If it has [t- but not cos/sin/^ => direct AR (X4[t-2])
        if '[t-' in expr and not any(func in expr for func in ['cos', 'sin', '^']):
            var_index = int(expr.split('X')[1].split('[')[0]) - 1
            lagval = parse_lag(expr.split('[')[1].split(']')[0])
            if t - lagval >= 0:
                return coef * X[t - lagval, var_index]
            else:
                return 0.0

        # 3) Nonlinear terms
        if 'cos(' in expr:
            # e.g. "cos(X1[t-2] * pi/2)"
            inner = expr.split('cos(')[1].split(')')[0]
            var_idx_ = int(inner.split('X')[1].split('[')[0]) - 1
            lag_str = inner.split('[')[1].split(']')[0]
            lagval = parse_lag(lag_str)
            val = X[t - lagval, var_idx_] if t - lagval >= 0 else 0.0
            return coef * np.cos(val * np.pi / 2)

        elif 'sin(' in expr:
            # e.g. "sin(X3[t-1] * pi/2)"
            inner = expr.split('sin(')[1].split(')')[0]
            var_idx_ = int(inner.split('X')[1].split('[')[0]) - 1
            lag_str = inner.split('[')[1].split(']')[0]
            lagval = parse_lag(lag_str)
            val = X[t - lagval, var_idx_] if t - lagval >= 0 else 0.0
            return coef * np.sin(val * np.pi / 2)

        elif '^' in expr:
            # e.g. "(X2[t])^3"
            power = int(expr.split('^')[1])
            var_idx_ = int(expr.split('X')[1].split('[')[0]) - 1
            if '[t]' in expr:
                val = X[t, var_idx_]
            else:
                lagval = parse_lag(expr.split('-')[1].split(']')[0]) if '[t-' in expr else 0
                val = X[t - lagval, var_idx_] if t - lagval >= 0 else 0.0
            return coef * (val ** power)

        # 4) Default linear e.g. "X2[t]"
        var_idx_ = int(expr.split('X')[1].split('[')[0]) - 1
        if '[t-' in expr:
            lagval = parse_lag(expr.split('[')[1].split(']')[0])
            val = X[t - lagval, var_idx_] if t - lagval >= 0 else 0.0
        else:
            # e.g. "X2[t]"
            val = X[t, var_idx_]
        return coef * val

    def generate_equations(self, t, X, U, trends, seasonality, n_vars, max_lag):
        noise = self.generate_noise(n_vars + 1)
        equations = get_nonlinear_equations(n_vars, max_lag)

        # Confounder first
        U[t] = noise[-1]
        var_values = {'U': U[t]}

        for eq in equations:
            if '=' not in eq or eq.startswith('U['):
                continue

            left, right = eq.split('=')
            left = left.strip()
            right = right.strip()

            var_name = left.split('[')[0]
            var_idx = int(var_name[1:]) - 1

            terms = right.split('+')
            val = 0.0

            for term in terms:
                term = term.strip()
                if term.startswith('e'):
                    val += noise[var_idx]
                elif term.startswith('trend'):
                    val += trends[var_idx][t]
                elif term.startswith('season'):
                    val += seasonality[var_idx][t]
                else:
                    # Evaluate e.g. "0.3 * sin(X3[t-1]*pi/2)"
                    val += self.evaluate_term(term, var_values, X, U, t)

            X[t, var_idx] = val
            var_values[var_name] = val

    def generate_multivariate_ts(self, n_points, n_vars, max_lag):
        X = np.zeros((n_points, n_vars))
        U = np.zeros(n_points)

        trends = [self.generate_trend(n_points, i) for i in range(n_vars)]
        seasonality = [self.generate_seasonality(n_points, i) for i in range(n_vars)]

        # Initialize first 'max_lag' steps
        for i in range(max_lag):
            X[i, :] = self.generate_noise(n_vars)
            U[i] = self.generate_noise(1)[0]
            for j in range(n_vars):
                X[i, j] += trends[j][i] + seasonality[j][i]

        # Then fill from t=max_lag onward
        for t in range(max_lag, n_points):
            self.generate_equations(t, X, U, trends, seasonality, n_vars, max_lag)

        df = pd.DataFrame(X, columns=[f"X{i+1}" for i in range(n_vars)])
        df["U"] = U
        df["time"] = np.arange(n_points)
        return df


# Helper Functions for Graph & Saving

def extract_causal_links(equations):
    links = {}
    for eq in equations:
        if '=' not in eq or eq.startswith('U['):
            continue
        left, right = eq.split('=')
        left = left.strip()
        right = right.strip()
        target = f"X{int(left.split('X')[1].split('[')[0])}"

        for term in right.split('+'):
            term = term.strip()
            if 'U' in term:
                coef = float(term.split('*')[0])
                links[('U', 0, target, 'confounder')] = coef
                continue

            if 'X' not in term:
                continue

            coef = float(term.split('*')[0])
            if 'cos(' in term:
                func = 'cos'
                source_var = int(term.split('X')[1].split('[')[0])
                source = f"X{source_var}"
            elif 'sin(' in term:
                func = 'sin'
                source_var = int(term.split('X')[1].split('[')[0])
                source = f"X{source_var}"
            elif '^' in term:
                func = f"power{term.split('^')[1]}"
                source_var = int(term.split('X')[1].split('[')[0])
                source = f"X{source_var}"
            else:
                func = 'linear'
                source_var = int(term.split('X')[1].split('[')[0])
                source = f"X{source_var}"

            if '[t-' in term:
                lag = int(term.split('-')[1].split(']')[0])
            else:
                lag = 0

            links[(source, lag, target, func)] = coef

    return links

def save_dataset_and_graph(df, n_vars, max_lag, sample_size, output_dir="output_nonlinear"):
    Path(output_dir).mkdir(parents=True, exist_ok=True)
    base_filename = f"{output_dir}/nonlinear_ts_with_confounder_n{sample_size}_vars{n_vars}_lag{max_lag}"

    # 1) Save CSV
    df.to_csv(f"{base_filename}.csv", index=False)

    # 2) Build adjacency from equations
    equations = get_nonlinear_equations(n_vars, max_lag)
    true_links = extract_causal_links(equations)

    var_names = [f"X{i+1}" for i in range(n_vars)] + ["U"]
    n_total_vars = n_vars + 1
    val_matrix = np.zeros((n_total_vars, n_total_vars, max_lag + 1))
    graph_matrix = np.zeros((n_total_vars, n_total_vars, max_lag + 1), dtype=bool)

    for (source, lag, target, func), weight in true_links.items():
        if source == 'U':
            source_idx = n_vars
        else:
            source_idx = int(source[1:]) - 1
        if target == 'U':
            target_idx = n_vars
        else:
            target_idx = int(target[1:]) - 1

        val_matrix[source_idx, target_idx, abs(lag)] = weight
        graph_matrix[source_idx, target_idx, abs(lag)] = True
        if lag == 0:
            val_matrix[target_idx, source_idx, 0] = weight

    # Plot adjacency
    plt.figure(figsize=(12, 12))
    tp.plot_time_series_graph(
        val_matrix=val_matrix,
        graph=graph_matrix,
        var_names=var_names,
        link_colorbar_label='Nonlinear Effect Strength',
        node_size=0.05
    )
    plt.title(f'Nonlinear Causal Graph with Confounder\n(n={sample_size}, vars={n_vars}, lag={max_lag})')
    plt.savefig(f'{base_filename}_graph.png')
    plt.close()

    # Plot time series
    plt.figure(figsize=(15, 10))
    for col in df.columns[:-2]:  # exclude U and time
        plt.plot(df['time'], df[col], label=col, alpha=0.7)
    plt.title(f'Nonlinear Time Series with Confounder\n(n={sample_size}, vars={n_vars}, lag={max_lag})')
    plt.xlabel('Time')
    plt.ylabel('Value')
    plt.legend()
    plt.grid(True)
    plt.savefig(f'{base_filename}_series.png')
    plt.close()

    # Decomposition (simple trend overlay)
    plt.figure(figsize=(15, 15))
    n_cols = min(3, n_vars)
    n_rows = (n_vars + n_cols - 1) // n_cols
    for i, col in enumerate(df.columns[:-2]):
        plt.subplot(n_rows, n_cols, i+1)
        plt.plot(df['time'], df[col], label='Series', alpha=0.7)
        z = np.polyfit(df['time'], df[col], 1)
        p = np.poly1d(z)
        plt.plot(df['time'], p(df['time']), "r--", label='Trend')
        plt.title(f'{col} Components')
        plt.legend()
        plt.grid(True)
    plt.tight_layout()
    plt.savefig(f'{base_filename}_decomposition.png')
    plt.close()

    # Save structure
    with open(f'{base_filename}_structure.txt', 'w') as f:
        f.write("Nonlinear Causal Structure with Confounder:\n\n")
        f.write("Equations:\n")
        for eq in equations:
            f.write(f"{eq}\n")
        f.write("\nCausal Links:\n")
        for (source, lag, target, func), coef in true_links.items():
            f.write(f"{source} --({func}, lag={lag})--> {target}: {coef}\n")



# Generate Multiple Combinations

def generate_all_combinations():
    sample_sizes = [500, 1000, 3000]
    n_vars_list = [4, 6, 8]
    max_lags = [2, 3, 4]
    trend_strengths = [0.01, 0.02]
    seasonal_strengths = [0.3, 0.5]

    for n in sample_sizes:
        for vars_ in n_vars_list:
            for lag in max_lags:
                for trend_str in trend_strengths:
                    for seas_str in seasonal_strengths:
                        print(f"\nGenerating dataset: n={n}, vars={vars_}, lag={lag}, trend={trend_str}, seas={seas_str}")
                        generator = NonlinearTimeSeriesGenerator(
                            noise_scale=0.1,
                            trend_strength=trend_str,
                            seasonal_strength=seas_str,
                            seasonal_period=12,
                            random_state=42
                        )
                        df = generator.generate_multivariate_ts(
                            n_points=n,
                            n_vars=vars_,
                            max_lag=lag
                        )
                        save_dataset_and_graph(df, vars_, lag, n)
                        print(" -> Saved successfully!")



#  Main

if __name__ == "__main__":
    print("Generating nonlinear time series with safe lag parsing...")

    # Example usage
    sample_sizes = [500, 1000, 3000, 5000]
    n_vars_list = [4, 6, 8]
    max_lags = [2, 3, 4]

    for n in sample_sizes:
        for vars_ in n_vars_list:
            for lag in max_lags:
                print(f"\nGenerating dataset: n={n}, vars={vars_}, lag={lag}")
                gen = NonlinearTimeSeriesGenerator(
                    noise_scale=0.1,
                    trend_strength=0.01,
                    seasonal_strength=0.5,
                    seasonal_period=12,
                    random_state=42
                )
                df = gen.generate_multivariate_ts(
                    n_points=n,
                    n_vars=vars_,
                    max_lag=lag
                )
                save_dataset_and_graph(df, vars_, lag, n)
                print(f"Dataset and plots saved for n_vars={vars_}, lag={lag}.")

    print("\nAll done!")


Generating nonlinear time series with safe lag parsing...

Generating dataset: n=500, vars=4, lag=2
Dataset and plots saved for n_vars=4, lag=2.

Generating dataset: n=500, vars=4, lag=3
Dataset and plots saved for n_vars=4, lag=3.

Generating dataset: n=500, vars=4, lag=4
Dataset and plots saved for n_vars=4, lag=4.

Generating dataset: n=500, vars=6, lag=2
Dataset and plots saved for n_vars=6, lag=2.

Generating dataset: n=500, vars=6, lag=3
Dataset and plots saved for n_vars=6, lag=3.

Generating dataset: n=500, vars=6, lag=4
Dataset and plots saved for n_vars=6, lag=4.

Generating dataset: n=500, vars=8, lag=2
Dataset and plots saved for n_vars=8, lag=2.

Generating dataset: n=500, vars=8, lag=3
Dataset and plots saved for n_vars=8, lag=3.

Generating dataset: n=500, vars=8, lag=4
Dataset and plots saved for n_vars=8, lag=4.

Generating dataset: n=1000, vars=4, lag=2
Dataset and plots saved for n_vars=4, lag=2.

Generating dataset: n=1000, vars=4, lag=3
Dataset and plots saved for 

/usr/local/lib/python3.11/dist-packages/tigramite/plotting.py:3203: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig = pyplot.figure(figsize=figsize)
<ipython-input-13-11f8f39240fe>:376: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  plt.figure(figsize=(15, 10))
<ipython-input-13-11f8f39240fe>:388: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly clo

Dataset and plots saved for n_vars=4, lag=3.

Generating dataset: n=3000, vars=4, lag=4


<ipython-input-13-11f8f39240fe>:363: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  plt.figure(figsize=(12, 12))


Dataset and plots saved for n_vars=4, lag=4.

Generating dataset: n=3000, vars=6, lag=2
Dataset and plots saved for n_vars=6, lag=2.

Generating dataset: n=3000, vars=6, lag=3
Dataset and plots saved for n_vars=6, lag=3.

Generating dataset: n=3000, vars=6, lag=4
Dataset and plots saved for n_vars=6, lag=4.

Generating dataset: n=3000, vars=8, lag=2
Dataset and plots saved for n_vars=8, lag=2.

Generating dataset: n=3000, vars=8, lag=3
Dataset and plots saved for n_vars=8, lag=3.

Generating dataset: n=3000, vars=8, lag=4
Dataset and plots saved for n_vars=8, lag=4.

Generating dataset: n=5000, vars=4, lag=2
Dataset and plots saved for n_vars=4, lag=2.

Generating dataset: n=5000, vars=4, lag=3
Dataset and plots saved for n_vars=4, lag=3.

Generating dataset: n=5000, vars=4, lag=4
Dataset and plots saved for n_vars=4, lag=4.

Generating dataset: n=5000, vars=6, lag=2
Dataset and plots saved for n_vars=6, lag=2.

Generating dataset: n=5000, vars=6, lag=3
Dataset and plots saved for n_va

...

<Figure size 1200x1200 with 0 Axes>

<Figure size 1200x1200 with 0 Axes>

<Figure size 1200x1200 with 0 Axes>

<Figure size 1200x1200 with 0 Axes>

<Figure size 1200x1200 with 0 Axes>

In [14]:
# Optional: Zip and download output
!zip -r /content/output_nonlinear_with_confounder.zip /content/output_nonlinear
from google.colab import files
files.download('/content/output_nonlinear_with_confounder.zip')

  adding: content/output_nonlinear/ (stored 0%)
  adding: content/output_nonlinear/nonlinear_ts_with_confounder_n500_vars8_lag3_structure.txt (deflated 62%)
  adding: content/output_nonlinear/nonlinear_ts_with_confounder_n1000_vars4_lag2.csv (deflated 50%)
  adding: content/output_nonlinear/nonlinear_ts_with_confounder_n500_vars4_lag4.csv (deflated 50%)
  adding: content/output_nonlinear/nonlinear_ts_with_confounder_n3000_vars6_lag4_structure.txt (deflated 59%)
  adding: content/output_nonlinear/nonlinear_ts_with_confounder_n500_vars6_lag3_graph.png (deflated 4%)
  adding: content/output_nonlinear/nonlinear_ts_with_confounder_n1000_vars8_lag2_series.png (deflated 6%)
  adding: content/output_nonlinear/nonlinear_ts_with_confounder_n1000_vars6_lag2_decomposition.png (deflated 4%)
  adding: content/output_nonlinear/nonlinear_ts_with_confounder_n500_vars8_lag2_decomposition.png (deflated 4%)
  adding: content/output_nonlinear/nonlinear_ts_with_confounder_n3000_vars8_lag3_series.png (deflat

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>